<a href="https://colab.research.google.com/github/jcfellers/461-fall-2020/blob/master/JCFellers_FinalDT_Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Justin Fellers - Assignment 1 Abstract 

I've been programming in python for about 4 months now, starting just at the end last semester.  This is my first significant implementation using a Class and a nested dictionary. This exercise really stretched my mind and I learned so much.  In addition to your lecture notes and the video you posted, I also consulted the following references, with number 4 being the one I got the most out of to construct my code in a similar fashion:

1) https://blog.clairvoyantsoft.com/entropy-information-gain-and-gini-index-the-crux-of-a-decision-tree-99d0cdc699f4

2) https://scikit-learn.org/0.15/modules/generated/sklearn.cross_validation.train_test_split.html

3) https://medium.com/@dhiraj8899/decision-tree-from-scratch-in-python-629631ec3e3a

4) https://medium.com/swlh/decision-tree-implementation-from-scratch-in-python-1cff4c00c71f

Up until now all of my coding has been in python IDLE and Spyder, which is where I coded this and why it is in a single cell.  I can send you my .py if you prefer and I will learn Jupyter/Colab before the next assignment.

The outline of the code is as follows.  The DTClassifer object is created with a user-parameter for max_depth of the tree.  The when fitted with an input X and input Y, the object concatenates the two into 'df' for manipulation.  The object then goes into building the tree with the '_build_tree(df)' operation. Then the following steps/substeps occur:

  1) produce the feature to serve as the node and the threshold to separate continuous features by using '_choose_feature_split(df)': 
  
  1a) calculate the df entropy by using '_calc_parent_entropy(df)'. The df's entropy value is returned. 

  1b) calculate the entropies of all features by using '_calc_feature_entropy(df, feature)'. The entropy and threshold of each feature are returned. 
  
  1b1) if the feature is categorical, the entropy of each feature value is computed then muliplied by the weight factor (feature value count/total df count).  These are the summands of the feature's total entropy. 

  1b2) if the feature is numerical, the feature is split at a threshold value assigned as the median value of the df's feature.  The left child is for features <= the threshold, and the right child is for the features > the threshold.  The entropies of the two children are the summands for the feature's total entropy.  

  1c) the '_choose_feature_split(df)' calculates the information gain for each feature by subracting its entropy from that of the parent entropy. The feature which produces the highest information gain and the threshold are returned to  '_build_tree(df)'

  2a) With feature/threshold returned, if the feature is categorical then a subset of records from df is created by '_split_categorical_feature(df, feature, v)'. If this subset only has 1 of the target values in it, this indicates a homogenous target set and the target value it is added to the tree as a leaf value with the feature value serving as the node.  Otherwise, this subset is recursed back into '_build_tree(df)' as the df.  A counter keeps track of the depth, and if it is equal to the max depth parameter the target which appears most in the subset is assigned as the leaf.  

  2b) If the feature is numeric, two subsets are created: one for the left child where the feature is <= threshold and the other for the right child where the feature is > threshold.  For the left subset and for the right subset, both are evaluated for homogenous target values to be append as a leaf or recursively enter '_build_tree(df)' the same as 2a.  

  3a) Once the tree is built, predictions for each row in the input X are made. Each row and the constructed tree are input to '_predict_target_value(row, tree)'. 

  3b) For each node of the tree, the imported row is assigned to a variable that represents its branch assignment.  This branch is then read back into the function to start over with the next node, until the branch reaches its leaf value.  The eventual leaf value is the predicted value for the row.  

  4) After all the predictions have been made, they are stored in the Series 'prediction_results'.  

  5) To calculate the accuracy, the number of times a row's actual target is equal to the predicted target is counted and divided by the number of actual targets. 


Below are further explanations about using the algorithm on the respective datasets.  Thank you for your time in reviewing my work and I look forward to hearing your feedback.  I'm a novice to programming and ML but want to keep learning.  



------------------------------------------------------------------------------
Titanic Dataset Discussion


The code starts by importing the dataset and seeing if there is any column where each record is a unique value.  If so, it drops the column.  This prevents the algorithm from memorizing the dataset and overfitting.  For this dataset, the 'Names' column is dropped.  For the titanic dataset I convert the 'Pclass' datatype into a string so it may be considered as a category.  One issue I ran into with this algorithm is that certain subsets never distill down to a single target.  I noticed this with the 'Pclass' in particular.  For example, say 'Age' is chosen first as the feature to split on and the threshold is 28.  Then 'Pclass' is chosen next.  There is a liklihood both target values will be the subset without another other feature to further split on, such as when the 'Fare' is 0 the family categories are also 0.  This leads the DT to run indefinitely, which I know from lecture is not supposed to occur.  I rectified this issue with a bypass where if the same subet of rows occur back to back, the target which is most frequent in the subset is chosen as the leaf value.  This  prevents the tree from running indefinitely if given a max_depth of None and works decently well for its intended purpose, as shown by an accuracy score of 98% on training data when allowed to run without a maximum depth.   

The results of this tree on this dataset are interesting, as it appears the tree is just as effective at making predictions with a depth of 1 as it is with any value of increased depth.  The feature 'Fare' is nearly always selected as the first split, and when conducting an analysis to determine the best depth the results for the train and test set display minimal variance.  I believe this is connected to the tree's inability to distill down to every record given the design I have implemented.  Experimenting with the size of the tree depth also highlighted an error with the dictionary method when predicting the test set.  On occasion, there is a node in the tree that does not split on a feature value that is in a test record.  For example, the fitted tree may split first on 'Fare', with children nodes under a threshold and above a threshold value.  Then say the right child splits on a 'Pclass' value of 3 and the left child splits on a 'Pclass' value of 1.  If the test record has a value of 2, then the dictionary returns a KeyError.  I could not implement a fix to this issue, although noticed it rarely due to the high frequency at which other Features are split on before 'Pclass' is used.  To prevent this error from crashing the script, I instituted an exception with a returned prediciton of randomly choosing one of the target values.  

As can be seen in the print window below, I experimented with four different depth sizes: 10, 20, 30, 40.  I'm not sure if this an extensive enough range, but I think it is for a dataset with such few features.  For each one, a 80/20 split of train/test data is established and a tree is fitted with the train data.  Then the tree is tested for accuracy on the train set and the test set.  As can be seen, neither the accuracy for the training nor the testing sets vary.  What I should have seen here is an increase in training accuracy with a decrease in testing accuracy as the got larger.  Also shown are accuracy scores for training and testing without a maximum depth.  Note the high score on the training set to indicate overfitting, and although the accuracy score on the test set is higher I know the tree is overfit.  Whenever the KeyError bypass is implemented to avoid crashing the line prints, so in this case the last time I ran the script it occur one time in fitting the 5 trees of various depths and twice when fitting the tree with no maximum depth.  



In [21]:
import numpy as np
import pandas as pd
import random
from statistics import mean
from sklearn.model_selection import train_test_split

eps = np.finfo(float).eps

# decision tree classifer object
class DTClassifier:
    
    def __init__(self, max_depth = 50):
        self.depth = 0
        self.max_depth = max_depth      
    
    def fit(self, X, y):
        self.X_train = X
        self.y_train = y
        df = X.copy()
        df['Target'] = y.copy()
        self.tree = self._build_tree(df)
        
        #print ('\nDT(depth = %s : \n %s' % (self.depth, self.tree))

    def _build_tree(self, df, tree = None, prev_df = None):
        
        feature, threshold = self._choose_feature_split(df)
        #print(feature, threshold)
        
        if tree is None:
            tree = {}
            tree[feature] = {}
                    
        # Categorical Feature Build
        if threshold == None:
            datum = np.unique(df[feature])
            
            for v in datum:
                v_df = self._split_categorical_feature(df, feature, v)
                v_df_t, v_df_tc = np.unique(v_df['Target'], return_counts = True)
                
                if v_df.equals(prev_df):
                    tree[feature][v] = v_df_t[np.argmax(v_df_tc)]
                    #print('no improvement, assigned target')
                    break
                
                v2_df = v_df.copy()
                
                
                if len(v_df_t) == 1: 
                    tree[feature][v] = v_df_t[0]
                else:
                  self.depth += 1
                  
                  if self.max_depth is not None and self.depth > self.max_depth:
                      tree[feature][v] = v_df_t[np.argmax(v_df_tc)]
                      
                  else:
                      tree[feature][v] = self._build_tree(v_df, prev_df = v2_df)
                      
        # Numerical Feature Build - LC for <=, RC for > 
        else:
            lc_df, rc_df = self._split_numerical_feature(df, feature, threshold)
            
            lc_df_t, lc_df_tc = np.unique(lc_df['Target'], return_counts = True)
            rc_df_t, rc_df_tc = np.unique(rc_df['Target'], return_counts = True)
            #print(lc_df['Target'])
            if len(lc_df_t) == 1:
                tree[feature]['<=' + str(threshold)] = lc_df_t[0]
            else:
                self.depth += 1
                if self.max_depth is not None and self.depth > self.max_depth:
                    tree[feature]['<=' + str(threshold)] = lc_df_t[np.argmax(lc_df_tc)]
                else:
                    tree[feature]['<=' + str(threshold)] = self._build_tree(lc_df)
            
            if len(rc_df_t) == 1:
                tree[feature]['>' + str(threshold)] = rc_df_t[0]
            else:
                self.depth += 1
                if self.max_depth is not None and self.depth > self.max_depth:
                    tree[feature]['>' + str(threshold)] = rc_df_t[np.argmax(rc_df_tc)]
                else:
                    tree[feature]['>' + str(threshold)] = self._build_tree(rc_df)
           
            
        #print(tree.keys())
        return tree
                
##############################################################################

    def _split_categorical_feature(self, df, feature, feature_value):
        
        v_df = df[df[feature] == feature_value]
        #print(v_df)
        return v_df

##############################################################################

    def _split_numerical_feature(self, df, feature, threshold):
        
        lc_df = df[df[feature] <= threshold]
        rc_df = df[df[feature] > threshold]
        #print(lc_df, rc_df)
        return lc_df, rc_df
            
        
##############################################################################
        
        
    def _choose_feature_split(self, df):
        # lists for information gain values and thresholds
        info_gains = []
        thresholds = []
        featureLst = X.columns.to_list()
        for f in featureLst:
            parent_entropy = self._calc_parent_entropy(df) 
            feature_entropy, threshold = self._calc_feature_entropy(df, f)
            IG = parent_entropy - feature_entropy
            
            info_gains.append(IG)
            thresholds.append(threshold)
        
        feature = featureLst[np.argmax(info_gains)]
        threshold = thresholds[np.argmax(info_gains)]
                
        return feature, threshold
            

###############################################################################

    def _calc_parent_entropy(self, df):
        parent_entropy = 0
        datum = np.unique(df['Target'], return_counts = True)
        total = sum(datum[1])
                        
        for value in datum[0]:
            parent_entropy -= (datum[1][value]/total) * np.log2(datum[1][value]/total)
            
        return parent_entropy
                
    
##############################################################################
    
    def _calc_feature_entropy(self, df, feature):
        #feature = 'Pclass'
        feature_entropy = 0
        threshold = None
        
        # Categorical features
        if (df[feature].dtypes == object):
            datum = np.unique(df[feature])
            
            for v in datum:
                v_entropy = 0                
                v_df = df[df[feature] == v]
                v_count = len(v_df)
                #print(v_count)
                
                for t in np.unique(df['Target']):
                    t_df = df[df['Target'] == t]
                    v_t_count = len(t_df[t_df[feature] == v])
                    #print(v_t_count)
                    fraction = v_t_count / (v_count + eps)
                    if (fraction > 0):
                        v_entropy -= fraction * np.log2(fraction)
                        
                w = v_count/len(df)
                feature_entropy += w * v_entropy
            
            #print(feature_entropy)
        
        # Numerical features - split into >= median value as LC, < as RC
        elif (df[feature].dtypes != object):
            threshold = df[feature].median()
            lc_entropy = 0
            rc_entropy = 0            
            
            lc_df = df[df[feature] <= threshold]
            rc_df = df[df[feature] > threshold]
            lc_count = len(lc_df)
            rc_count = len(rc_df)

            for t in np.unique(df['Target']): 
                lc_t_count = len(lc_df[lc_df['Target'] == t])
                rc_t_count = len(rc_df[rc_df['Target'] == t])
                fraction_lc = lc_t_count/(lc_count + eps)
                fraction_rc = rc_t_count/(rc_count + eps)
                
                if (fraction_lc > 0):
                    lc_entropy -= fraction_lc * np.log2(fraction_lc)
                elif (fraction_rc > 0):
                    rc_entropy -= fraction_rc * np.log2(fraction_rc)
                      
            w_lc = lc_count/len(df)
            w_rc = rc_count/len(df)
            lc_entropy = w_lc * lc_entropy
            rc_entropy = w_rc * rc_entropy
            
            if (lc_entropy + rc_entropy) < 1:
                feature_entropy = lc_entropy + rc_entropy
            else:
                feature_entropy = 1
                
            #print(feature_entropy)
                
        return feature_entropy, threshold

##############################################################################
    
    def predict(self, X):
        results = []
        #feature_search = {}
        #print(X.index)
        #col_index = 0
        # for column in X.columns:
        #     feature_search[column] = col_index
        #     col_index += 1

        for i in X.index:
            #print (i)
            row_prediction = self._predict_target_value(X.loc[i], self.tree)
            results.append([i, row_prediction])
        
        results = pd.Series(results)
        #print(results)
        return results
        
                    
###############################################################################
    
    def _predict_target_value(self, row, tree):
        try:

            for node in tree.keys():
                result = row[node]
                
                if type(result) == str:
                    #print(result)
                    leaf = tree[node][result]
                else:
                    threshold = str(list(tree[node].keys())[0]).split('<=')[1]
                    #print('t is', threshold)
                    if result <= float(threshold):
                        leaf = tree[node]['<=' + str(threshold)]
                        
                    elif result > float(threshold):
                        leaf = tree[node]['>' + str(threshold)]
                        
                if type(leaf) is dict:
                    prediction = self._predict_target_value(row, leaf)
                    
                else:
                    prediction = leaf
                    
            return prediction

        except(KeyError):
          print('keyerror encountered, returned random target value')
          prediction = random.choice([0, 1])
          return prediction
###############################################################################
def calc_accuracy(y_actual, y_pred):
    #print(y_actual)
    
    #acc = number of correct predictions / number of predictions
    
    count_correct = 0
    for i in range(len(y_actual)):
        actual_target = y_actual.iloc[i]
        predicted_target = y_pred.iloc[i][1]
        
        if predicted_target == actual_target:
                count_correct += 1

    
    acc = round((count_correct/len(y_actual) * 100),2)
    
    return acc
    
############################################################################### 
    
if __name__ == '__main__':
    
    titanic_url = 'https://raw.githubusercontent.com/jcfellers/461-fall-2020/master/assignments/assignment-1/titanic.csv'

    data = pd.read_csv(titanic_url)
    
    # should any column be full of only unique values, drop it
    header = data.columns.values.tolist()

    for column in header:
      if len(data[column].unique()) == len(data):
        data = data.drop([column], axis = 1)
    
    # convert Pclass to object so it is categorical
    data['Pclass'] = data['Pclass'].astype(str)

    #print(data.head())

   	#Split Features and target
    X, y = data.drop(['Survived'], axis=1), data['Survived']
    #print(X.shape, y.shape)
    
    ###########################################################################
       
   # Search for best depth
    
    trainAcc = []
    testAcc = []

    for d in range(1, 50, 10):
        d_train_res = []
        d_test_res = []
        
        for t in range(5):
        
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)
        
            dt_clf = DTClassifier(max_depth = d)
        
            dt_clf.fit(X_train, y_train)
        
                  ##############################################
        
            prediction_results = dt_clf.predict(X_train)
        
            accuracy = calc_accuracy((y_train), prediction_results)
            d_train_res.append(accuracy)
        
            #print('\n d=%s, Train Accuracy: %s' % (d, accuracy))
        
            prediction_results = dt_clf.predict(X_test)
        
            accuracy = calc_accuracy((y_test), prediction_results)
            d_test_res.append(accuracy)
        
            #print('Test Accuracy: %s' % (accuracy))
            
        avg_d_train = round(mean(d_train_res), 2)
        avg_d_test = round(mean(d_test_res), 2)
        
        trainAcc.append(avg_d_train)
        testAcc.append(avg_d_test)
    
    print('Average training accuracy from 5 iterations by increasing depth left to right\n', trainAcc)
    print('Average testing accuracy from 5 iterations by increasing depth left to right\n', testAcc)

  ##############################################    
    print ('\n')
    print ('Train set and Test set accuracy when tree is given no maximum depth.')

    d = None

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

    dt_clf = DTClassifier(max_depth = d)

    dt_clf.fit(X_train, y_train)

    prediction_results = dt_clf.predict(X_train)

    accuracy = calc_accuracy((y_train), prediction_results)
    
    print('\n d=%s, Train Accuracy: %s' % (d, accuracy))

    prediction_results = dt_clf.predict(X_test)

    accuracy = calc_accuracy((y_test), prediction_results)

    print('\n d=%s, Test Accuracy: %s' % (d, accuracy))

keyerror encountered, returned random target value
Average training accuracy from 5 iterations by increasing depth left to right
 [64.12, 63.1, 64.57, 65.28, 66.46]
Average testing accuracy from 5 iterations by increasing depth left to right
 [61.68, 64.61, 65.06, 62.47, 62.81]


Train set and Test set accuracy when tree is given no maximum depth.

 d=None, Train Accuracy: 97.88
keyerror encountered, returned random target value
keyerror encountered, returned random target value

 d=None, Test Accuracy: 75.84


_______________________________________________________________________________
Cars Dataset Discussion

The cars dataset challenged the algorithm design and further showed the difficulties presented with different datatypes.  As discussed above with the titanic dataset, the decision tree had complications with the categorical data types, but was mostly alleviated by the tree's ability to continually parse down the numerical types with different threshold values.  

In the cars dataset, I first added the feature names to the dataset prior to importing it.  Then I changed all of the features to integer types between 0 and 3 for those with four feature values, and 0 and 2 for those with three feature values.  This allowed me to take advantage of the numerical threshold process that proved more effective with the titanic dataset, but presented a new issue because the feature values, while numbers, do not represent continuous features.  I had to engineer workarounds to allow node creations for branches that led no where in order to get the script to run.  In larger trees this led to more frequent KeyErrors than in the titanic dataset when predicting test data.  

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
 data = pd.read_csv('gdrive/My Drive/car.data')
 data.head()

,buying,maint,doors,persons,lug_boot,safety,car
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [19]:
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from statistics import mean

eps = np.finfo(float).eps

# decision tree classifer object
class DTClassifier:
    
    def __init__(self, max_depth = None):
        
        self.depth = 0
        self.max_depth = max_depth
        self.features = list
        self.X_train = np.array        
        self.y_train = np.array        
        self.num_features = int        
        self.train_size = int
        
    
    def fit(self, X, y):
        
        self.X_train = X
        self.y_train = y
        self.features = list(X.columns)
        self.train_size = X.shape[0]
        self.num_features = X.shape[1]
        
        # df is the X and y together, features and target, for analyses
        df = X.copy()
        df['Target'] = y.copy()
                
        # Step 1: object is constructed from fit function; begins building tree
        self.tree = self._build_tree(df)
        
        #print ('\nDT(depth = %s : \n %s' % (self.depth, self.tree))


    def _build_tree(self, df, tree = None, prev_df = None):
        
        feature, threshold = self._choose_feature_split(df)
        #print(feature, threshold)
        
        if tree is None:
            tree = {}
            tree[feature] = {}
                    
        # Categorical Feature Build
        if threshold == None:
            datum = np.unique(df[feature])
            
            for v in datum:
                v_df = self._split_categorical_feature(df, feature, v)               
                v_df_t, v_df_tc = np.unique(v_df['Target'], return_counts = True)
                
                if v_df.equals(prev_df):
                    tree[feature][v] = v_df_t[np.argmax(v_df_tc)]
                    break
                
                v2_df = v_df.copy()
                
                if len(v_df_t) == 1: 
                    tree[feature][v] = v_df_t[0]
                else:
                  self.depth += 1
                  
                  if self.max_depth is not None and self.depth > self.max_depth:
                      tree[feature][v] = v_df_t[np.argmax(v_df_tc)]
                      
                  else:
                      tree[feature][v] = self._build_tree(v_df, prev_df = v2_df)
                      
        # Numerical Feature Build - LC for <=, RC for > 
        else:
            lc_df, rc_df = self._split_numerical_feature(df, feature, threshold)
            
            lc_df_t, lc_df_tc = np.unique(lc_df['Target'], return_counts = True)
            rc_df_t, rc_df_tc = np.unique(rc_df['Target'], return_counts = True)
            
            if len(lc_df_t) == 0:
                tree[feature]['<=' + str(threshold)] = None
            if len(rc_df_t) == 0:
                tree[feature]['>' + str(threshold)] = None
                
            else:
                #print(lc_df['Target'])
                if len(lc_df_t) == 1:
                    tree[feature]['<=' + str(threshold)] = lc_df_t[0]
                else:
                    self.depth += 1
                    if self.max_depth is not None and self.depth > self.max_depth:
                        tree[feature]['<=' + str(threshold)] = lc_df_t[np.argmax(lc_df_tc)]
                    else:
                        tree[feature]['<=' + str(threshold)] = self._build_tree(lc_df)
                
                if len(rc_df_t) == 1:
                    tree[feature]['>' + str(threshold)] = rc_df_t[0]
                else:
                    self.depth += 1
                    if self.max_depth is not None and self.depth > self.max_depth:
                        tree[feature]['>' + str(threshold)] = rc_df_t[np.argmax(rc_df_tc)]
                    else:
                        tree[feature]['>' + str(threshold)] = self._build_tree(rc_df)
           
            
        #print(tree.keys())
        return tree
                
##############################################################################

    def _split_categorical_feature(self, df, feature, feature_value):
        
        v_df = df[df[feature] == feature_value]
        #print('vDF', v_df)
        return v_df

##############################################################################

    def _split_numerical_feature(self, df, feature, threshold):
        
        lc_df = df[df[feature] <= threshold]
        rc_df = df[df[feature] > threshold]
        #print(lc_df, rc_df)
        return lc_df, rc_df
            
        
##############################################################################
        
        
    def _choose_feature_split(self, df):
        # lists for information gain values and thresholds
        info_gains = []
        thresholds = []
        featureLst = X.columns.to_list()
        for f in featureLst:
            parent_entropy = self._calc_parent_entropy(df) 
            feature_entropy, threshold = self._calc_feature_entropy(df, f)
            IG = parent_entropy - feature_entropy
            
            info_gains.append(IG)
            thresholds.append(threshold)
        
        feature = featureLst[np.argmax(info_gains)]
        threshold = thresholds[np.argmax(info_gains)]
                
        return feature, threshold


###############################################################################

    def _calc_parent_entropy(self, df):
        parent_entropy = 0
        datum = np.unique(df['Target'], return_counts = True)
        total = sum(datum[1])
        #print(datum)
                        
        # for value in datum[0]:
        #     parent_entropy -= (datum[1][value]/total) * np.log2(datum[1][value]/total)
        
        for i in range(len(datum[0])):
            parent_entropy -= (datum[1][i]/total) * np.log2(datum[1][i]/total)
        return parent_entropy
                
    
##############################################################################
    
    def _calc_feature_entropy(self, df, feature):
        #feature = 'Pclass'
        feature_entropy = 0
        threshold = None
        
        # Categorical features
        if (df[feature].dtypes == object):
            datum = np.unique(df[feature])
            
            for v in datum:
                v_entropy = 0                
                v_df = df[df[feature] == v]
                v_count = len(v_df)
                #print(v_count)
                
                for t in np.unique(df['Target']):
                    t_df = df[df['Target'] == t]
                    v_t_count = len(t_df[t_df[feature] == v])
                    #print(v_t_count)
                    fraction = v_t_count / (v_count + eps)
                    if (fraction > 0):
                        v_entropy -= fraction * np.log2(fraction)
                        
                w = v_count/len(df)
                feature_entropy += w * v_entropy
            
            #print(feature_entropy)
        
        # Numerical features - split into >= median value as LC, < as RC
        elif (df[feature].dtypes != object):
            threshold = df[feature].median()
            lc_entropy = 0
            rc_entropy = 0            
            
            lc_df = df[df[feature] <= threshold]
            rc_df = df[df[feature] > threshold]
            lc_count = len(lc_df)
            rc_count = len(rc_df)

            for t in np.unique(df['Target']): 
                lc_t_count = len(lc_df[lc_df['Target'] == t])
                rc_t_count = len(rc_df[rc_df['Target'] == t])
                fraction_lc = lc_t_count/(lc_count + eps)
                fraction_rc = rc_t_count/(rc_count + eps)
                
                if (fraction_lc > 0):
                    lc_entropy -= fraction_lc * np.log2(fraction_lc)
                elif (fraction_rc > 0):
                    rc_entropy -= fraction_rc * np.log2(fraction_rc)
                      
            w_lc = lc_count/len(df)
            w_rc = rc_count/len(df)
            lc_entropy = w_lc * lc_entropy
            rc_entropy = w_rc * rc_entropy
            
            if (lc_entropy + rc_entropy) < 1:
                feature_entropy = lc_entropy + rc_entropy
            else:
                feature_entropy = 1
                
            #print(feature_entropy)
                
        return feature_entropy, threshold

##############################################################################
    
    def predict(self, X):
        results = []
        #feature_search = {}
        #print(X.index)
        #col_index = 0
        # for column in X.columns:
        #     feature_search[column] = col_index
        #     col_index += 1

        for i in X.index:
            
            #print(X.loc[i])
            row_prediction = self._predict_target_value(X.loc[i], self.tree)
            results.append([i, row_prediction])
        
        results = pd.Series(results)
        #print(results)
        return results
        
                    
###############################################################################
    
    def _predict_target_value(self, row, tree):
        try:

            for node in tree.keys():
                result = row[node]
                
                if type(result) == str:
                    #print(result)
                    leaf = tree[node][result]
                else:
                    #print(str(list(tree[node].keys())[0]))
                    
                        
                    try:
                        threshold = str(list(tree[node].keys())[0]).split('<=')[1]
                        
                    except:
                        threshold = str(list(tree[node].keys())[0]).split('>')[1]
                        
                    
                    # if threshold == None:
                    #     break
                    
                    #print('t is', threshold)
                    if result <= float(threshold):
                        leaf = tree[node]['<=' + str(threshold)]
                        
                    elif result > float(threshold):
                        leaf = tree[node]['>' + str(threshold)]
                        
                if type(leaf) is dict:
                    prediction = self._predict_target_value(row, leaf)
                    
                else:
                    prediction = leaf
                    
            return prediction

        except(KeyError):
          print('keyerror encountered, returned random target value')
          prediction = random.choice(['acc', 'unacc', 'good', 'vgood'])
          return prediction
###############################################################################
def calc_accuracy(y_actual, y_pred):
    #print(y_actual)
    
    #acc = number of correct predictions / number of predictions
    
    count_correct = 0
    for i in range(len(y_actual)):
        actual_target = y_actual.iloc[i]
        predicted_target = y_pred.iloc[i][1]
        
        if predicted_target == actual_target:
                count_correct += 1

    
    acc = round((count_correct/len(y_actual) * 100),2)
    
    return acc
    
############################################################################### 
    
if __name__ == '__main__':
    
    print('\n-------------- Dataset------------------------')
    
    data = pd.read_csv('gdrive/My Drive/car.data')
    
    # should any column be full of only unique values, drop it
    header = data.columns.values.tolist()

    for column in header:
      if len(data[column].unique()) == len(data):
        data = data.drop([column], axis = 1)
      
    
    data['buying'].value_counts()
    for i in range(len(data)):
        if data['buying'].iloc[i] == 'vhigh':
            data['buying'].iloc[i] = 3
        if data['buying'].iloc[i] == 'high':
            data['buying'].iloc[i] = 2
        if data['buying'].iloc[i] == 'med':
            data['buying'].iloc[i] = 1
        if data['buying'].iloc[i] == 'low':
            data['buying'].iloc[i] = 0
    data['buying'] = data['buying'].astype(int)
    
    data['maint'].value_counts()
    for i in range(len(data)):
        if data['maint'].iloc[i] == 'vhigh':
            data['maint'].iloc[i] = 3
        if data['maint'].iloc[i] == 'high':
            data['maint'].iloc[i] = 2
        if data['maint'].iloc[i] == 'med':
            data['maint'].iloc[i] = 1
        if data['maint'].iloc[i] == 'low':
            data['maint'].iloc[i] = 0
    data['maint'] = data['maint'].astype(int)
        
    data['doors'].value_counts()
    for i in range(len(data)):
        if data['doors'].iloc[i] == '5more':
            data['doors'].iloc[i] = 3
        if data['doors'].iloc[i] == '4':
            data['doors'].iloc[i] = 2
        if data['doors'].iloc[i] == '3':
            data['doors'].iloc[i] = 1
        if data['doors'].iloc[i] == '2':
            data['doors'].iloc[i] = 0
    data['doors'] = data['doors'].astype(int)
        
    data['persons'].value_counts()
    for i in range(len(data)):
        if data['persons'].iloc[i] == 'more':
            data['persons'].iloc[i] = 2
        if data['persons'].iloc[i] == '4':
            data['persons'].iloc[i] = 1
        if data['persons'].iloc[i] == '2':
            data['persons'].iloc[i] = 0  
    data['persons'] = data['persons'].astype(int)
        
    data['lug_boot'].value_counts()
    for i in range(len(data)):
        if data['lug_boot'].iloc[i] == 'big':
            data['lug_boot'].iloc[i] = 2
        if data['lug_boot'].iloc[i] == 'med':
            data['lug_boot'].iloc[i] = 1
        if data['lug_boot'].iloc[i] == 'small':
            data['lug_boot'].iloc[i] = 0  
    data['lug_boot'] = data['lug_boot'].astype(int)
        
    data['safety'].value_counts()
    for i in range(len(data)):
        if data['safety'].iloc[i] == 'high':
            data['safety'].iloc[i] = 2
        if data['safety'].iloc[i] == 'med':
            data['safety'].iloc[i] = 1
        if data['safety'].iloc[i] == 'low':
            data['safety'].iloc[i] = 0 
    data['safety'] = data['safety'].astype(int)
            
   	#Split Features and target
    X, y = data.drop(['car'], axis=1), data['car']
    # print(X.head(), y.head())
    X.describe()

    ###########################################################################
    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)
    # dt_clf = DTClassifier(max_depth = 5)
        
    # dt_clf.fit(X_train, y_train)
     
    # prediction_results = dt_clf.predict(X_train)
    # accuracy = calc_accuracy((y_train), prediction_results)
    # print('\nTrain Accuracy: %s' % accuracy)
    
    # accuracy = calc_accuracy((y_test), prediction_results)        
    # print('Test Accuracy: %s' % (accuracy))
    #Search for best depth
    
    y.value_counts()
    trainAcc = []
    testAcc = []

    for d in range(1, 15, 5):
        d_train_res = []
        d_test_res = []
        
        for t in range(5):
        
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)
        
            dt_clf = DTClassifier(max_depth = d)
        
            dt_clf.fit(X_train, y_train)
        
                  ##############################################
        
            prediction_results = dt_clf.predict(X_train)
        
            accuracy = calc_accuracy((y_train), prediction_results)
            d_train_res.append(accuracy)
        
            #print('\n d=%s, Train Accuracy: %s' % (d, accuracy))
        
            prediction_results = dt_clf.predict(X_test)
        
            accuracy = calc_accuracy((y_test), prediction_results)
            d_test_res.append(accuracy)
        
            #print('Test Accuracy: %s' % (accuracy))
            
        avg_d_train = round(mean(d_train_res), 2)
        avg_d_test = round(mean(d_test_res), 2)
        
        trainAcc.append(avg_d_train)
        testAcc.append(avg_d_test)
    print('\n')
    print('Average training accuracy from 5 iterations by increasing depth left to right\n', trainAcc)
    print('Average testing accuracy from 5 iterations by increasing depth left to right\n', testAcc)


-------------- Dataset------------------------


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)




Average training accuracy from 5 iterations by increasing depth left to right
 [70.27, 70.15, 70.8]
Average testing accuracy from 5 iterations by increasing depth left to right
 [69.02, 69.42, 70.98]


_______________________________________________________________________________
Conclusion

This was an extremely challenging assignment for me, to which I invested way more time than I anticipated.  In doing so however, I learned way more about decision trees than expected and have a profound appreciation for the power of a robustly created algorithm that can handled data of all types across large feature sets.  I understand the theory of the decision tree, how to calculate and select the feature with the highest information gain and how this reduces impurity.  My frustrations and shortcomings with implementation largely stem from construction the "physical structure" that is the tree.  

Thank you for your time.

- Justin